# Dashboards

So far we have looked over some of the web-based technologies for generating interactive elements. We specifically looked at `hvPlot` and `panel`. The former provides a separate but very similar plotting interface from `pandas` that provides us with many convenient add-ons. The latter allows us to create interactive widgets that we can use to control and modify our visualizations. Both of these tools are mostly built on top of `bokeh`.

In this notebook we will look at more examples of how we can start constructing basic dashboards with `bokeh`/`hvPlot`/`holoviews` and `panel`.

## Imports

First thing we need to so is import a few things to get started:

In [1]:
import hvplot.pandas  # for plotting from data frames
import numpy as np  # for data generation
import pandas as pd  # for data frames
import panel as pn  # for interactive widgets

note that we do not need to import `bokeh` explicitly - `hvplot` and `panel` handle all of this for us! We may need to use `bokeh` directly, but for now we do not.

## Generating Data

Now we need to work with some data! We are just going to randomly generate some basic data that consists of a label, an id (x value) and a value (y value). We want data that is basic enough for a scatter plot, but also clearly groupable (in our case, the label). We also ensure that the x values across the entire set are already sorted and uniformly distributed.

In [2]:
# Create state for random number generator for consistency
rng = np.random.RandomState(13337)

# Create a DataFrame from a dictionary

# Create the label column by randomly selecting values from the list (given to
# a) 100 times, using the probabilities/weights given for each.
label_choices = rng.choice(
    a=["foo", "bar", "baz"], p=[0.5, 0.35, 0.15], size=100
)

# Create the ID column by making a list of integers 0-99
ids = np.arange(0, 100, 1)

# Create the value column by selecting 100 numbers between 0.0 and 1.0
values = rng.uniform(size=100)

data = pd.DataFrame(
    {
        "label": label_choices,
        "id": ids,
        "value": values,
    }
)

data

,label,id,value
0,bar,0,0.355499
1,baz,1,0.042054
2,foo,2,0.372780
3,foo,3,0.833910
4,baz,4,0.238624
...,...,...,...
95,bar,95,0.422872
96,bar,96,0.555540
97,foo,97,0.864862
98,bar,98,0.433510


## Widgets

We are going to create some widgets that allow us to select labels of data and we will dynamically update our visualization with this data. We also want to provide a threshold filter (we saw something similar last class) to "disable" values. The widget we will use for selecting labels is called a `MultiSelect`, and the widget we will use for filtering values is called a `FloatSlider`.

In [3]:
# Create a MultiSelect widget
label_selector = pn.widgets.MultiSelect(
    name="Label",  # Name of the widget
    options=["foo", "bar", "baz"],  # Options for users to select from
    value=["foo", "bar", "baz"],  # Default selections
)

# Create a FloatSLider
threshold_slider = pn.widgets.FloatSlider(
    name="Threshold",  # Name of the widget
    start=0.0,  # Lower bound of the slider
    end=1.0,  # Upper bound of the slider
    step=0.01,  # Step value
    value=1.0,  # Default value of the slider
)
label_selector

BokehModel(combine_events=True, render_bundle={'docs_json': {'0e56df59-2fce-4f5b-950a-6249f226398c': {'version…

In [4]:
threshold_slider

BokehModel(combine_events=True, render_bundle={'docs_json': {'7a703891-c8e9-48c6-bc70-71532d82151c': {'version…

In subsequent cells below we are going to be recreating the the widgets in each cell that we would need them. As we will be make multiple plots with the same widgets, we do not want the rendered widgets from one cell to modify the plots from the other cells. This will happen because the state of the widgets will be tied to the state of the plots.

## Connecting Plots & Widgets

Now we need to create a function that plots our data. It needs to be a function because as the widgets change they need to reproduce the plot. They can only do this by giving it a set of instructions to carry out, which is exactly what a function is! There are a few different ways we can do this, but we are just going to lean into `panel` to handle this for us using the `interact` functionality.

The first thing we need to do is write a function that plots our data using values we expect to give it from our widgets. We have widgets for selecting labels and setting thresholds, and so our function should expect to take two variables - one for each of them. This function should build a mask from the two variables, apply it to the data frame, and then use `hvPlot` to scatter the data.

We anchor the limits of our axes for smoother behavior.

In [5]:
# First define a function that takes in placeholders for our widgets
def visualize_data(selections, threshold):
    # Create the mask by checking which labels are in our selections
    # and which values are less than or equal to our threshold
    mask = (data.label.isin(selections)) & (data.value <= threshold)

    updated_plot = (
        data[mask]
        .hvplot.scatter(  # Apply the mask and scatter via hvPlot
            x="id",  # Put the id on the x-axis
            y="value",  # Put the value on the y-xis
            by="label",  # Group by labels (i.e. colo by label)
        )
        .opts(  # Further customize the plot via its options (opts)
            xlim=(0, 100),  # Set the range of the x-axis to 0-100
            ylim=(0, 1),  # Set the range of the y-axis to 0-1
        )
    )

    return updated_plot


# Repeat our widgets
label_selector = pn.widgets.MultiSelect(
    name="Label", options=["foo", "bar", "baz"], value=["foo", "bar", "baz"]
)
threshold_slider = pn.widgets.FloatSlider(
    name="Threshold", start=0.0, end=1.0, step=0.01, value=1.0
)

# Use panel to stitch our functions together with our widgets, mapping
# placeholders to widgets
interaction = pn.interact(
    visualize_data, selections=label_selector, threshold=threshold_slider
)
interaction

BokehModel(combine_events=True, render_bundle={'docs_json': {'da871cec-5a28-4961-9ab9-308afcdbd9ff': {'version…

The problem here is that if we deselect all of our labels, then not only is there nothing to plot, but `hvPLot` does not like it when it attempts to plot groups of data (as specified by `by='label'` and there is nothing to group. It is generally a good practice to check if there is actual data to plot, and in the case where there is nothing to plot we instead return a *pane* that says so.

In [6]:
# Updated function
def visualize_data(selections, threshold):
    mask = (data.label.isin(selections)) & (data.value <= threshold)

    # Instead of just plotting here, we first check if the mask is too strict.
    #
    # If the dataframe with the mask is empty, then return a message. Otherwise
    # plot as we did before.
    if data[mask].empty:
        message = (
            "# No Data to Plot!\n### Perhaps you filtered too aggressively"
        )
        pane_with_message = pn.pane.Markdown(message)
        return pane_with_message
    else:
        updated_plot = (
            data[mask]
            .hvplot.scatter(x="id", y="value", by="label")
            .opts(xlim=(0, 100), ylim=(0, 1))
        )
        return updated_plot


label_selector = pn.widgets.MultiSelect(
    name="Label", options=["foo", "bar", "baz"], value=["foo", "bar", "baz"]
)
threshold_slider = pn.widgets.FloatSlider(
    name="Threshold", start=0.0, end=1.0, step=0.01, value=1.0
)

interaction = pn.interact(
    visualize_data, selections=label_selector, threshold=threshold_slider
)
interaction

BokehModel(combine_events=True, render_bundle={'docs_json': {'c01e8b64-d63a-4874-b762-d8f463186c02': {'version…

A problem here, though this works, is that the colors are not consistent. As we develop interactive displays we need to pay attention to what details should be changing and what details actually are changing. Here `bokeh` (for better or worse) alphabetizes the labels used for grouping the data, and thus depending on the selections different colors are used for different labels.

Here we need to rework the scatter a bit, providing it with a color map that maps labels to specific colors. This means that as users provide different selections, the labels retain their specific colors.

In [7]:
# Create a dictionary that maps label names to colors
color_mapper = {"foo": "red", "bar": "green", "baz": "blue"}


# Updated function
def visualize_data(selections, threshold):
    mask = (data.label.isin(selections)) & (data.value <= threshold)
    if data[mask].empty:
        message = (
            "# No Data to Plot!\n### Perhaps you filtered too aggressively"
        )
        pane_with_message = pn.pane.Markdown(message)
        return pane_with_message
    else:
        updated_plot = (
            data[mask]
            .hvplot.scatter(
                x="id",
                y="value",
                color="label",  # use the color specifier instead of by
                cmap=color_mapper, # provide our dictionary as the color_mapper
            )
            .opts(xlim=(0, 100), ylim=(0, 1))
        )
        return updated_plot


label_selector = pn.widgets.MultiSelect(
    name="Label", options=["foo", "bar", "baz"], value=["foo", "bar", "baz"]
)
threshold_slider = pn.widgets.FloatSlider(
    name="Threshold", start=0.0, end=1.0, step=0.01, value=1.0
)

interaction = pn.interact(
    visualize_data, selections=label_selector, threshold=threshold_slider
)
interaction

BokehModel(combine_events=True, render_bundle={'docs_json': {'35977421-081a-47bd-a457-8d9ff7087b4d': {'version…